### library import
---
사용 할 패키지를 미리 가져옵니다.

In [59]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

### 데이터 전처리
---
영화이름이 적힌 데이터와 영화시청 데이터를 각각 받은 뒤, movie_id수와 title수를 비교하여 동명의 영화존재여부를 확인했습니다.\
동명의 영화가 존재하지 않는 다는 것을 확인한 뒤, 두 데이터를 합치고, 사용하는 것만 남겼습니다.

In [60]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [61]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [62]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [63]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [64]:
data=pd.merge(ratings, movies, on='movie_id')
using_cols = ['user_id', 'title', 'counts']
data=data[using_cols]
data.head()

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5


### 데이터 탐색
---
ratings에 있는 유니크한 영화 개수\
ratings에 있는 유니크한 사용자 수\
가장 인기 있는 영화 30개(인기순)\
에 대한 정보를 확인했습니다.


In [65]:
# data['movie_id'].nunique()
data['title'].nunique()

3628

In [66]:
data['user_id'].nunique()

6039

In [67]:
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 사용자 초기정보입력
---
선호하는 영화를 5가지 골라서 ratings에 추가했습니다.\
추가하는 과정에서 인덱싱이 잘 못되서 실행을 몇 번 반복했지만, 코드 상의 문제를 발견하기 전에 제대로 실행됬습니다

In [68]:
# 본인이 좋아하는 아티스트 데이터로 바꿔서 추가하세요.
my_favorite = ['Jurassic Park (1993)', 'Back to the Future (1985)', 'Braveheart (1995)', 'Godfather, The (1972)', 'Forrest Gump (1994)']

# 'skc'이라는 user_id가 위 아티스트의 오래를 30회씩 들었다고 가정합시다.
my_playlist = pd.DataFrame({'user_id' : ['skc']*5, 'title':my_favorite, 'counts' : [4.0]*5})

# user_id에 'skc'이라는 데이터가 없다면 위에 임의로 만든 my_favortie 데이터를 추가해줍시다.
if not data.isin({'user_id' : ['skc']})['user_id'].any():
    data = data.append(my_playlist)
    
data.tail(10)

,user_id,title,counts
836473,5851,One Little Indian (1973),5.0
836474,5854,Slaughterhouse (1987),4.0
836475,5854,"Promise, The (Versprechen, Das) (1994)",3.0
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4.0
836477,5948,Identification of a Woman (Identificazione di ...,5.0
0,skc,Jurassic Park (1993),4.0
1,skc,Back to the Future (1985),4.0
2,skc,Braveheart (1995),4.0
3,skc,"Godfather, The (1972)",4.0
4,skc,Forrest Gump (1994),4.0


In [69]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
title_to_idx = {v : k for k, v in enumerate(title_unique)}

In [70]:
data.tail(10)

,user_id,title,counts
836473,5851,One Little Indian (1973),5.0
836474,5854,Slaughterhouse (1987),4.0
836475,5854,"Promise, The (Versprechen, Das) (1994)",3.0
836476,5938,"Five Wives, Three Secretaries and Me (1998)",4.0
836477,5948,Identification of a Woman (Identificazione di ...,5.0
0,skc,Jurassic Park (1993),4.0
1,skc,Back to the Future (1985),4.0
2,skc,Braveheart (1995),4.0
3,skc,"Godfather, The (1972)",4.0
4,skc,Forrest Gump (1994),4.0


In [71]:
print(user_to_idx['skc'])
print(title_to_idx['Braveheart (1995)'])

6039
87


In [72]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(data): # 모든 row가 정상적으로 인덱싱 되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data # data['user_id']을 인덱싱된 Series로 교체해줍니다.
else:
    print("user_id column indexing fail!!")
    
# artist_to_idx을 통해 artist 칼럼도 동일한 방식으로 인덱싱 해줍니다.
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing fail!!')
    
data.head(10)

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5.0
1,1,0,5.0
2,2,0,4.0
3,3,0,4.0
4,4,0,5.0
5,5,0,4.0
6,6,0,5.0
7,7,0,5.0
8,8,0,3.0
9,9,0,5.0


### CSR matrix 생성
---
음악데이터와 비슷한 방식으로 csr matrix를 생성했습니다만, 다른 방식으로 생성하는 방법도 알아둬야 할 것 같습니다. 

In [73]:
num_user = data['user_id'].nunique()
num_artist = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_artist))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### als_model 구성 및 훈련
---
Martix Factorization 모델을 implicit패키지를 이용하여 학습했습니다.\
MF에서 두 Feature Matrix를 한꺼번에 학습하는 것은 잘 수렴하지 않아서,\
번갈아 가며 학습하는 방식의 als_model이 효과적이라고 합니다.

parameter 중 factors 와 iterations를 늘릴 시 학습도가 올라가지만, 과적합의 원인이 되기도합니다.

In [74]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [76]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [77]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [78]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 선호도 확인
---
선호하는 영화와 그렇지 않은 모델을 선정하여 선호도를 확인했습니다.\
선호하는 영화는 0.6 초반의 값을 선호하지 않는 영화는 0.2 초반의 값을 보였습니다.\
주어진 데이터에 비해 0.6 초반의 값은 부족한 값이 아닌가하는 생각을 갖고 있습니다.

In [81]:
skc, brave_heart = user_to_idx['skc'], title_to_idx['Braveheart (1995)']
skc_vector, brave_heart_vector = als_model.user_factors[skc], als_model.item_factors[brave_heart]

print(np.dot(skc_vector, brave_heart_vector))

0.639825


In [82]:
american_beauty = title_to_idx['American Beauty (1999)']
american_beauty_vector = als_model.item_factors[american_beauty]

print(np.dot(skc_vector, american_beauty_vector))

0.23430586


### 비슷한 영화 추천
---
음악추천 함수를 이용하여 영화추천을 했습니다.\
함수안에 dict자료와 index dict자료를 추가한 것 외의 변형은 없습니다.\

In [89]:
def get_similar_title(title: str):
    title_id = title_to_idx[title]
    similar_title = als_model.similar_items(title_id)
    
    idx_to_title = {v:k for k,v in title_to_idx.items()}
    [idx_to_title[i[0]] for i in similar_title]
    
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title




In [91]:
get_similar_title('Jurassic Park (1993)')

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Lost World: Jurassic Park, The (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Braveheart (1995)',
 'Fifth Element, The (1997)',
 'Little Odessa (1994)']

### 유저에게 영화 추천
---
als 모델내에 이미 있는 recommend 메소드를 이용하여 좋아 할 만한 영화를 추천 받도록 했습니다.\
filter_already_liked_items은 유저가 평가한 영화를 제외하는 parameter입니다.

유저가 평가한 영화가 추천에 영향을 미치는 정도를 explain메소드를 활용하여 확인했습니다.

In [93]:
user = user_to_idx['skc']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_title[i[0]] for i in title_recommended]

['Godfather: Part II, The (1974)',
 'Saving Private Ryan (1998)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Godfather: Part III, The (1990)',
 'Matrix, The (1999)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'American Beauty (1999)',
 'Groundhog Day (1993)',
 'Dances with Wolves (1990)',
 'Independence Day (ID4) (1996)',
 'Rocky Horror Picture Show, The (1975)',
 'Boat, The (Das Boot) (1981)',
 'Legends of the Fall (1994)',
 'Last of the Mohicans, The (1992)',
 "Schindler's List (1993)",
 'Airplane! (1980)',
 'Princess Bride, The (1987)']

In [95]:
terminator = title_to_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_data, itemid=terminator)

[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Jurassic Park (1993)', 0.23726221234362777),
 ('Braveheart (1995)', 0.1075625998907844),
 ('Back to the Future (1985)', 0.035544932123658),
 ('Godfather, The (1972)', 0.004342283035292665),
 ('Forrest Gump (1994)', -0.011673535906903532)]

### 회고
---
완성하는데, 급급하여 노드와 유사한 방식으로만 마무리를 하였지만,\
데이터에 주어졌던, genre등의 데이터 역시 같은 방식으로 활용하였다면 더 훌륭한 추천방식이 되지 않았을까 생각합니다\
